# 🤖 CareBot with Zep + AutoGen
This notebook demonstrates how to build a compassionate mental health bot with memory using Microsoft AutoGen and Zep.

### 📦 Step 1: Install dependencies
Run this in a terminal before running the notebook:

In [ ]:
!pip install autogen~=0.3 zep-cloud python-dotenv

### 🔐 Step 2: Setup API Keys
Create a `.env` file in the same folder with:
```env
OPENAI_API_KEY=your_openai_key
ZEP_API_KEY=your_zep_key
```

In [ ]:
import os
import uuid
import asyncio
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
ZEP_API_KEY = os.getenv("ZEP_API_KEY")

config_list = [{
    "model": "gpt-4o",
    "api_key": OPENAI_API_KEY,
    "base_url": "https://api.openai.com/v1"
}]


In [ ]:
carebot_system_message = """
You are a compassionate mental health bot and caregiver. Review information about the user and their prior conversation below and respond accordingly.
Keep responses empathetic and supportive. And remember, always prioritize the user's well-being and mental health. Keep your responses very concise and to the point.
"""

In [ ]:
from autogen import ConversableAgent
from zep_cloud.client import AsyncZep
from zep_cloud import Message

class ZepConversableAgent(ConversableAgent):
    def __init__(self, name, zep_memory, zep_session_id, *args, **kwargs):
        super().__init__(name, *args, **kwargs)
        self.zep_memory = zep_memory
        self.zep_session_id = zep_session_id

    async def generate_reply(self, messages, sender, config=None):
        for message in messages:
            role = "user" if message["role"] == "user" else "assistant"
            await self.zep_memory.add_memory(
                session_id=self.zep_session_id,
                message=Message(role=role, content=message["content"])
            )
        memory = await self.zep_memory.search_memory(
            session_id=self.zep_session_id,
            text=messages[-1]["content"],
            metadata={"agent": self.name}
        )
        memory_text = "\n".join([m.content for m in memory]) if memory else ""
        if memory_text:
            self.system_message += f"\n\nRelevant memory:\n{memory_text}"
        return await super().generate_reply(messages, sender, config)


In [ ]:
zep = AsyncZep(api_key=ZEP_API_KEY)
session_id = str(uuid.uuid4())
bot_name = "CareBot"

agent = ZepConversableAgent(
    bot_name,
    zep_memory=zep,
    zep_session_id=session_id,
    system_message=carebot_system_message,
    llm_config={"config_list": config_list},
    function_map=None,
    human_input_mode="NEVER",
)


In [ ]:
messages = [{"role": "user", "content": "I’m feeling very anxious today and can’t focus on anything."}]

async def run_agent():
    reply = await agent.generate_reply(messages, sender="User")
    print("CareBot reply:", reply)

await run_agent()